In [10]:

def carregar_sequencia(filename):
    from Bio import SeqIO
    
    # Lemos o ficheiro .fasta
    seq_proteina = SeqIO.read(open(filename),format='fasta')

    sp = seq_proteina.seq # Isolamos a sequência

    return sp

In [6]:
# Procedemos à contagem de aminoácidos:
# Primeiro para a sequência traduzida e depois para o conjunto de sequências após a remoção de codões stop

sp = carregar_sequencia('output\\WQB05986.fasta')

def conta_aa(seq_prot):
    # TODO colocar um dicionário bonitinho
    from collections import Counter
    contagem_aa = dict(Counter(seq_prot))
    
    return contagem_aa

conta_aa(sp)


{'M': 6,
 'I': 13,
 'L': 26,
 'N': 11,
 'K': 10,
 'A': 15,
 'G': 22,
 'T': 16,
 'V': 23,
 'S': 20,
 'P': 15,
 'C': 5,
 'E': 16,
 'D': 10,
 'H': 7,
 'Y': 7,
 'Q': 9,
 'F': 13,
 'R': 6,
 'W': 5}

In [7]:
swiss_id = 'P01909' # Encontra-se o ID no ficheiro FASTA 

def swiss_prot_scan(swiss_id):
    from Bio import SeqIO
    from Bio import ExPASy
    # Efetuamos uma pesquisa na base de dados SwissProt 
    handle = ExPASy.get_sprot_raw(swiss_id) # Encontramos o ID ao ler o ficheiro fasta
    sr = SeqIO.read(handle, "swiss")
    print(
        f'ID {sr.id}',
        f'Sequência: {sr.seq}',
        f'Tamanho da sequência: {len(sr.seq)} bp',
        f'Nome: {sr.name}',
        f'Descrição: {sr.description}',
        f'Taxonomia: {sr.annotations["taxonomy"]}',
        f'Organismo: {sr.annotations["organism"]}',
        f'Keywords: {sr.annotations["keywords"]}',
        sep = '\n')

swiss_prot_scan('P01909')

ID P01909
Sequência: MILNKALMLGALALTTVMSPCGGEDIVADHVASYGVNLYQSYGPSGQYTHEFDGDEQFYVDLGRKETVWCLPVLRQFRFDPQFALTNIAVLKHNLNSLIKRSNSTAATNEVPEVTVFSKSPVTLGQPNILICLVDNIFPPVVNITWLSNGHSVTEGVSETSFLSKSDHSFFKISYLTLLPSAEESYDCKVEHWGLDKPLLKHWEPEIPAPMSELTETVVCALGLSVGLVGIVVGTVFIIRGLRSVGASRHQGPL
Tamanho da sequência: 254 bp
Nome: DQA1_HUMAN
Descrição: RecName: Full=HLA class II histocompatibility antigen, DQ alpha 1 chain; AltName: Full=DC-1 alpha chain; AltName: Full=DC-alpha; AltName: Full=HLA-DCA; AltName: Full=MHC class II DQA1; Flags: Precursor;
Taxonomia: ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
Organismo: Homo sapiens (Human)
Keywords: ['3D-structure', 'Adaptive immunity', 'Cell membrane', 'Direct protein sequencing', 'Disulfide bond', 'Endoplasmic reticulum', 'Endosome', 'Glycoprotein', 'Golgi apparatus', 'Immunity', 'Lysosome', 'Membrane', 'MHC II', 'Refer

In [14]:
def analise_blast(email, sequencia_proteina, filename, lim = None):
    import os
    from Bio.Blast import NCBIWWW
    
    NCBIWWW.email = email

    
    
    # Verifica se já existe um ficheiro XML para os resultados do BLAST. Ignora a pesquisa caso já exista.
    # Para poupar tempo.
    # Se o ficheiro já existir executa o parsing do ficheiro.

    if '.xml' not in filename:
        filename = filename + '.xml'

    if os.path.isfile(filename): 
        
        blast_parse(filename,lim)

    else: 
    # Executa o BLAST da Proteína
        handle = NCBIWWW.qblast('blastp','swissprot',sequencia_proteina.seq)

        with open(filename, 'w') as _:
            _.write(handle.read())
        
        handle.close()

analise_blast('pg52648@alunos.uminho.pt', "WQB05986.fasta" ,'blast_results.xml')

AttributeError: 'str' object has no attribute 'seq'

In [31]:
filename = 'blast_results.xml'

def blast_parse(filename, lim = None):
    
    from Bio.Blast import NCBIXML

    handle = open(filename)
    blast_res = NCBIXML.parse(handle)
    blast_records = list(blast_res)

    num = 1
    for record in blast_records:

        if lim == None: lim = len(record.alignments)    

        for aligment in record.alignments[0:lim]:
            #print(aligment)
            
            for hsp in aligment.hsps:
                #print(hsp)
                
                if hsp.expect < 0.05:
                    print('****Alinhamento {} ****'.format(num))
                    print('Sequência:',aligment.title)
                    print('Tamanho da Sequência:',aligment.length)
                    print('e-value:',hsp.expect)
                    print(hsp.query[0:40]+'...')
                    print(hsp.match[0:40]+'...')
                    print(hsp.sbjct[0:40]+'...')
                    print()
                    num += 1    
